In [399]:
import datetime as datetime

SESSION = init_fetch_session()

In [ ]:
gwo_price = 13.58
gwo_spread = 130
gwo_reset_date = datetime.datetime(2020,12,31)
gwo_dividend_annual = 0.544

estimated_goc5_at_reset_decimal = 0.02

resets_maturity_date = datetime.datetime(2040,12,15)
gwo_months = [3, 6, 9, 12]



gwo_reset_dividend_annual = dividend_after_reset(gwo_spread, estimated_goc5_at_reset_decimal)




In [396]:
# Next Steps
#
# 1. read in CSV table of fixed resets; update prices
# 2. compute the current market spread
# 3. compute future dividend (add to table)
# 4. given a market spread, compute price as a function of various GOC5 estimate
# 5. loop over multiple MS scenarios. Unlike specifying explicitly, use functional guess


resets = pd.read_csv('./resets.csv', dtype={'Notes': object})
resets.drop(columns=['Notes','RefPrice'],inplace=True,errors='ignore')

df = resets.copy()
TICKER_BLACKLIST = ['BBD.PR.D', 'GMP.PR.B', 'BIR.PR.A','CSE.PR.A']

# remove unacceptable tickers
df = df[[x not in TICKER_BLACKLIST for x in df['Ticker']]]



In [ ]:
df = update_data_frame_with_prices_and_drop_reference(df, SESSION)

In [405]:

# 2. Compute current market spread
update_dataframe_with_reset_market_spread(df, 0.0166)


# Current yield

,Ticker,Rating,Div,Spread,Reset Date,Price,MSpread
0,AIM.PR.A,ZR,1.12500,375,03/2020,16.95,0.0498
1,AIM.PR.C,ZR,1.50250,420,03/2024,19.49,0.0605
2,BAM.PR.G,P2L,0.68750,230,11/2021,14.50,0.0308
3,EMA.PR.A,P3H,0.63875,184,08/2020,14.50,0.0275
4,PWF.PR.P,P2H,0.57650,160,01/2021,13.75,0.0253
...,...,...,...,...,...,...,...
169,RY.PR.H,P2In,0.91250,226,08/2024,17.90,0.0344
170,BMO.PR.T,P2In,0.90600,224,08/2024,17.60,0.0349
171,FTS.PR.M,P3H,1.02500,248,12/2024,17.95,0.0405
172,TRP.PR.B,P2L,0.53800,128,06/2020,12.10,0.0279


In [437]:




# Compute current yield
df["CurYield"] = [ div/price for (div,price) in zip(df["Div"],df["Price"])]

# 3. Compute future dividend
df["FutureDiv"] = [dividend_from_reference_and_issue_spread( 1.66,issue_spread) 
                   for (issue_spread) in df["Spread"]]
# 3b. Future current yield (which won't last, but just for fun)



df["FutCurYield"] = [ div/price for (div,price) in zip(df["FutureDiv"],df["Price"])]




# 4. Given a market spread delta and a GOC5, compute expected price. Price
# is clamped to 25.5

df["FutPrice"] = [ share_price_given_ref_rate_and_market_spread(1.60,
                                                               mspread*100,
                                                               irs)
                  for (mspread,irs) in zip(df["MSpread"],df["Spread"])]

# Gain in decimal (x 100 for percdnt)
df["CapGainDec"] = [ futprice/curprice-1 for (futprice,curprice) in 
                     zip(df["FutPrice"], df["Price"])]

df["Duration"] = [ year_frac_from_today(dstr) for dstr in df["Reset Date"]]

## compute annualized capgain
df["AnnGain"] = [ (( (capgain+1) ** (1/num_years))-1 ) for (capgain,num_years) in
                zip(df["CapGainDec"],df["Duration"])]

df.sort_values(by="AnnGain",ascending=False)



,Ticker,Rating,Div,Spread,Reset Date,Price,MSpread,Duration,CurYield,FutureDiv,FutCurYield,FutPrice,CapGain,AnnGain,CapGainDec
165,TD.PF.C,P2In,0.9375,225,01/2020,17.95,0.0356,0.005479,0.052228,0.9775,0.054457,18.653101,1.039170,1108.945140,0.039170
160,CM.PR.P,P2In,0.9375,224,01/2020,17.30,0.0376,0.005479,0.054191,0.9750,0.056358,17.910448,1.035286,559.418018,0.035286
150,FFH.PR.E,P3H,0.7275,216,03/2020,14.87,0.0323,0.169863,0.048924,0.9550,0.064223,19.461698,1.308789,3.875549,0.308789
0,AIM.PR.A,ZR,1.1250,375,03/2020,16.95,0.0498,0.169863,0.066372,1.3525,0.079794,20.326748,1.199218,1.913945,0.199218
142,BRF.PR.A,P3H,0.8388,262,04/2020,17.03,0.0327,0.254795,0.049254,1.0700,0.062830,21.663244,1.272064,1.571397,0.272064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,ENB.PF.E,P3H,1.1000,266,06/2020,16.74,0.0491,0.421918,0.065711,1.0800,0.064516,16.359447,0.977267,-0.053044,-0.022733
48,BAM.PR.T,P2L,1.1250,231,03/2022,16.20,0.0528,2.169863,0.069444,0.9925,0.061265,14.207849,0.877028,-0.058680,-0.122972
158,ENB.PF.C,P3H,1.1000,264,03/2020,16.74,0.0491,0.169863,0.065711,1.0750,0.064217,16.282642,0.972679,-0.150478,-0.027321
157,BAM.PF.E,P2L,1.1000,255,03/2020,17.63,0.0458,0.169863,0.062394,1.0525,0.059699,16.788026,0.952242,-0.250307,-0.047758


In [440]:
df.sort_values(by="AnnGain",ascending=False).head(10)

,Ticker,Rating,Div,Spread,Reset Date,Price,MSpread,Duration,CurYield,FutureDiv,FutCurYield,FutPrice,CapGain,AnnGain,CapGainDec
165,TD.PF.C,P2In,0.9375,225,01/2020,17.95,0.0356,0.005479,0.052228,0.9775,0.054457,18.653101,1.039170,1108.945140,0.039170
160,CM.PR.P,P2In,0.9375,224,01/2020,17.30,0.0376,0.005479,0.054191,0.9750,0.056358,17.910448,1.035286,559.418018,0.035286
150,FFH.PR.E,P3H,0.7275,216,03/2020,14.87,0.0323,0.169863,0.048924,0.9550,0.064223,19.461698,1.308789,3.875549,0.308789
0,AIM.PR.A,ZR,1.1250,375,03/2020,16.95,0.0498,0.169863,0.066372,1.3525,0.079794,20.326748,1.199218,1.913945,0.199218
142,BRF.PR.A,P3H,0.8388,262,04/2020,17.03,0.0327,0.254795,0.049254,1.0700,0.062830,21.663244,1.272064,1.571397,0.272064
66,FFH.PR.M,P3H,1.1875,398,03/2020,21.95,0.0375,0.169863,0.054100,1.4100,0.064237,25.750000,1.173121,1.559925,0.173121
38,HSE.PR.E,P2L,1.1250,357,03/2020,18.98,0.0427,0.169863,0.059273,1.3075,0.068888,22.018739,1.160102,1.397152,0.160102
172,TRP.PR.B,P2L,0.5380,128,06/2020,12.10,0.0279,0.421918,0.044463,0.7350,0.060744,16.400911,1.355447,1.056141,0.355447
151,SLF.PR.G,P1L,0.5688,141,06/2020,13.60,0.0252,0.421918,0.041824,0.7675,0.056434,18.264563,1.342983,1.011608,0.342983
133,RY.PR.J,P2Hn,0.9000,274,05/2020,20.16,0.0280,0.336986,0.044643,1.1000,0.054563,24.659091,1.223169,0.818086,0.223169


In [432]:
# TRP.PR.B
# print(share_price_given_ref_rate_and_market_spread(1.6,  2.8, 128))
df

,Ticker,Rating,Div,Spread,Reset Date,Price,MSpread,Duration,CurYield,FutureDiv,FutCurYield,FutPrice,CapGain,AnnGain
0,AIM.PR.A,ZR,1.12500,375,03/2020,16.95,0.0498,0.131507,0.066372,1.3525,0.079794,20.326748,1.199218,2.980659
1,AIM.PR.C,ZR,1.50250,420,03/2024,19.49,0.0605,4.134247,0.077091,1.4650,0.075167,18.954248,0.972511,-0.006719
2,BAM.PR.G,P2L,0.68750,230,11/2021,14.50,0.0308,1.802740,0.047414,0.9900,0.068276,20.833333,1.436782,0.222662
3,EMA.PR.A,P3H,0.63875,184,08/2020,14.50,0.0275,0.550685,0.044052,0.8750,0.060345,19.770115,1.363456,0.755892
4,PWF.PR.P,P2H,0.57650,160,01/2021,13.75,0.0253,0.969863,0.041927,0.8150,0.059273,19.370460,1.408761,0.423843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,RY.PR.H,P2In,0.91250,226,08/2024,17.90,0.0344,4.553425,0.050978,0.9800,0.054749,19.146825,1.069655,0.014898
170,BMO.PR.T,P2In,0.90600,224,08/2024,17.60,0.0349,4.553425,0.051477,0.9750,0.055398,18.860511,1.071620,0.015307
171,FTS.PR.M,P3H,1.02500,248,12/2024,17.95,0.0405,4.887671,0.057103,1.0350,0.057660,18.053097,1.005744,0.001172
172,TRP.PR.B,P2L,0.53800,128,06/2020,12.10,0.0279,0.383562,0.044463,0.7350,0.060744,16.400911,1.355447,1.209827


In [261]:
MIN_GOC5_MARKET_SPREAD = 1.0


print(share_price_given_ref_rate_and_market_spread(1.60, 5.14, 375,
                                                   min_spread=MIN_GOC5_MARKET_SPREAD))



19.844213649851632


In [259]:
MINIMUM_TBILL_MARKET_SPREAD = 0.9


In [278]:
# the Saga of TRP.PR.B
# 
# now, we will do 3 GOC5 scenarios. Also do 3 spread scenarios: category mean, current MS, 
# and future MS (ie, assuming that the future yield is being used in current MS calc)
# Additional complication: do we use Hymas' perpectual YTW, or assume that maturity
# price will change based on market spread?
# 
# Also, shouldn't YTW be computed over the period of interest, rather than perpetuity?
#
# How about we do a "five year from now" maturity period?
#
# Let's do one example
#
# Assume GOC5 = 1.2%.   Reset date
# df.iloc[1]['Reset Date']


In [345]:
# Set up parameters
trp_reset_date = datetime.datetime(2020,6,1)
goc5_in_percent = 1.2
today = datetime.datetime.today()
eval_date = today+datetime.timedelta(days=365*5)
trp_cur_price = 12.10
trp_fut_price = 16.40
trp_div = 0.538
trp_fut_div = 0.735


# one day, consider that it could be a higher IRR to sell as soon as the capital
# gain occurs.  So one day should sweep the eval interval or at least check the
# reset date to see the highest.
    



In [348]:
flows = build_cashflow_list(cur_date, trp_cur_price, 
                            trp_reset_date, 
                            eval_date, trp_fut_price,
                            gwo_months,
                           trp_div, trp_fut_div)

In [349]:
xirr(flows, 0.2)

0.11862463969892087

In [350]:
alt_date = trp_reset_date + datetime.timedelta(days=30)

In [351]:
alt_date

datetime.datetime(2020, 7, 1, 0, 0)

In [352]:
flows2 = build_cashflow_list(cur_date, trp_cur_price, 
                            trp_reset_date, 
                            alt_date, trp_fut_price,
                            gwo_months,
                           trp_div, trp_fut_div)

In [353]:
flows2

[(datetime.datetime(2020, 1, 12, 20, 54, 1, 669511), -12.1),
 (datetime.datetime(2020, 3, 31, 0, 0), 0.1345),
 (datetime.datetime(2020, 6, 30, 0, 0), 0.18375),
 (datetime.datetime(2020, 7, 1, 0, 0), 16.4)]

In [354]:
xirr(flows2,0.1)

1.0087185914689318

In [377]:
import datetime as datetime


In [388]:
def year_frac_from_today(reset_date_str) :
    reset_date = csv_to_date(reset_date_str)
    today = datetime.datetime.today()
    time_diff = reset_date - today
    num_days = time_diff.days
    return num_days/365

In [390]:
foo = year_frac_from_today('03/2020')
foo

0.13150684931506848

In [387]:
foo.years

AttributeError: 'datetime.timedelta' object has no attribute 'years'

In [403]:
def fetch_last_close_in_dollars(standard_ticker, session ) :

    ticker = convert_ticker_to_yahoo(standard_ticker)
    print("Processing " + standard_ticker + ": " + ticker)
    
#    df = pdr.get_data_yahoo(ticker, start=datetime.datetime(2019,12,31),end= date.today())
    tmp_df = web.DataReader(ticker, 'yahoo', datetime.datetime(2019,12,31), 
                            datetime.datetime.today(), session=session)
    last_close = pd.Series(tmp_df['Adj Close'])[-1]
    return round(last_close,2)


In [434]:
def csv_to_date(dstr) :
    ds = dstr.split('/')  
    if len(ds[1]) < 4 :
        ds[1] = "20" + ds[1]
    isoformat = ds[1] + "-" + ds[0] + "-15"
    return datetime.datetime.fromisoformat(isoformat)


In [435]:
csv_to_date('03/24')

datetime.datetime(2024, 3, 15, 0, 0)

In [412]:
foo = "24"
len(foo)

2